# TIEnc

<div>
<img src="imgs/new_rotated.jpg" alt="Description" style="display: block; margin: 20px auto; width: 80%;" />
</div>

<!-- --- -->

| Generator | Validator | Status | Nodes | Chapters | Concepts | Relations | Prereqs | Part-Of | Defs | Props | Orphans | AvgRel/Node |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| claude-opus-4-5 | gpt5-1 | ✅ OK | 450 | 104 | 346 | 1395 | 71 | 632 | 30 | 143 | 0 | 3.1 |
| claude-opus-4-5 | claude-opus-4-5 | ✅ OK | 460 | 104 | 356 | 1432 | 69 | 600 | 26 | 198 | 0 | 3.11 |
| gpt5-1 | gpt5-1 | ✅ OK | 459 | 104 | 355 | 1726 | 119 | 536 | 209 | 242 | 0 | 3.76 |
| gpt5-1 | claude-opus-4-5 | ✅ OK | 423 | 104 | 319 | 1601 | 137 | 490 | 117 | 249 | 0 | 3.78 |


**Estruturas & Modelos**

In [3]:
from enum import Enum
from langchain.chat_models import init_chat_model, BaseChatModel

class LLM_MODELS(str, Enum):
    gpt4_o = "openai:gpt-4o"
    gpt5_1 = "openai:gpt-5.1"
    claude4_5 = "anthropic:claude-opus-4-5"

def get_llm(model_name: LLM_MODELS) -> BaseChatModel:
    return init_chat_model(model_name.value)

In [57]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

class Concept(BaseModel):
    """Represents a single educational concept found in the text."""
    concept_name: str = Field(description="The formal name of the concept (e.g., 'Inductive Logic', 'Backpropagation').")
    chapter: List[int] = Field(description="The number of the current chapter, subchapter, etc (e.g., [1] for chapter 1, [1, 2] for subchapter 1.2, [1,2,5] for subsubchapter 1.2.5)")
    description: str = Field(description="A concise definition or summary of the concept based on the text.")
    page_start: int = Field(description="The page number where this concept is first introduced.")
    # page_end: Optional[int] = Field(default=None, description="The page number where the discussion of this concept seems to end (or current page if ongoing).")
    # is_main_chapter: bool = Field(default=False, description="True if this is a chapter or main topic, False if it is a subchapter or subtopic.")

class PageExtraction(BaseModel):
    """Container for multiple concepts found on a specific page processing step."""
    concepts: List[Concept] = Field(description="List of concepts extracted from the current text window.")
    
class Relation(BaseModel):
    source: str = Field(description="The subject concept.")
    target: str = Field(description="The object concept.")
    # relation_type: Literal['prerequisite', 'including', 'part-of', 'property', 'definition']
    relation_type: Literal['prerequisite']
    context: Optional[str] = Field(description="Justification text.")

class ConceptAnalysis(BaseModel):
    """LLM Output for a full chapter/concept file."""
    # Map 'new_concepts' to add to registry
    # new_concepts: List[str] = Field(description="List of MAIN concepts defined in this text.")
    relations: List[Relation] = Field(description="Semantic connections found.")
    
class ValidationResult(BaseModel):
    valid_relations: List[Relation] = Field(description="The filtered list of strictly valid educational relations.")
    rejected_relations: List[Relation] = Field(description="List of relations that were removed.")

## Extração de dados

O SINKT já considera um dataset pronto para uso. Sendo assim essa seção busca extrair os conceitos de um ebook PDF. Primeiramente iremos transformar em Markdown, visto que é melhor utilizar texto puro ao invés de páginas de PDF. Além disso, essa proposta facilita a própria extração para o MAIC, posteriormente.

In [58]:
import unicodedata
import re
import xml.etree.ElementTree as ET
from xml.dom import minidom

def normalize_filename(s: str) -> str:
    """Remove accents, replace underscores and remove non-alphanumeric characters."""
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    s = re.sub(r'\s+', '_', s)
    s = re.sub(r'[^\w_]', '', s)
    return s.lower()

def prettify_xml(elem: ET.Element) -> str:
    """
    Return a pretty-printed XML string for the Element.
    Strips the annoying extra newlines minidom likes to add.
    
    :param elem: Element (``ET.Element``)
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    # Filter out lines that are purely whitespace
    return '\n'.join([line for line in reparsed.toprettyxml(indent="   ").split('\n') if line.strip()])

Configuração inicial.

In [6]:
from dotenv import load_dotenv
import os
from pathlib import Path

load_dotenv("../.env")

assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found"
assert os.getenv("ANTHROPIC_API_KEY"), "ANTHROPIC_API_KEY not found"

BOOK_NAME = 'LinuxFundamentals'
EBOOKS_PATH = Path('ebooks')
base_output_dir = EBOOKS_PATH / BOOK_NAME
os.makedirs(base_output_dir, exist_ok=True)

PDF_PATH = Path('../data/701-LinuxFundamentals_material_full_v14.pdf')

Primeiramente é criada a classe de conversão do PDF para markdown, utiliza-se da biblioteca Docling para realizar a conversão. Essa biblioteca permite extrair as imagens e tabelas do texto, posteriormente elas são incluídas no markdown final além de serem salvas juntas.

In [7]:
import os
from PyPDF2 import PdfReader

from docling.document_converter import DocumentConverter
import logging
from docling.datamodel.accelerator_options import AcceleratorDevice, AcceleratorOptions
from docling.datamodel.base_models import InputFormat, OutputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions
    )
from docling.document_converter import DocumentConverter, PdfFormatOption, MarkdownFormatOption
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem, DoclingDocument
from tqdm.notebook import tqdm_notebook

class PDFConversor():
    """
    Convert PDF to markdown.
    
    :param pdf_path: Path of the input pdf.
    :param output_dir: Path of the output.
    """
    def __init__(self, pdf_path: Path, output_dir: Path):
        self.input_doc_path: Path = pdf_path
        self.base_output_dir: Path = output_dir
        self.pipeline_options: PdfPipelineOptions = self._set_pipeline_options()
        self.document_converter: DocumentConverter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(pipeline_options=self.pipeline_options),
                OutputFormat.MARKDOWN: MarkdownFormatOption(image_mode=ImageRefMode.REFERENCED)
            },
        )
        self.last_page: int = self._get_no_pages()
        self.doc = None
       
    def _set_pipeline_options(self) -> PdfPipelineOptions:
        IMAGE_SCALE = 2.0
        
        pipeline_options = PdfPipelineOptions()
        pipeline_options.generate_picture_images = True
        pipeline_options.generate_page_images = True
        pipeline_options.images_scale = IMAGE_SCALE
        pipeline_options.do_ocr = False
        pipeline_options.do_table_structure = True
        pipeline_options.table_structure_options.do_cell_matching = True
        pipeline_options.ocr_options.lang = ["pt"]
        pipeline_options.accelerator_options = AcceleratorOptions(
            num_threads=4, device=AcceleratorDevice.CUDA
        )
        return pipeline_options

    
    def _get_no_pages(self) -> int:
        reader = PdfReader(self.input_doc_path)
        return len(reader.pages)
    
    def _replace_image_placeholders(selg, md_str: str, image_files: List[Path]) -> None:
        content = md_str
        for img in image_files:
            content = content.replace("<!-- image -->", f"![]({str(img).split('/')[-1]})", 1)
        return content
        
    def save_images(self, doc: DoclingDocument, output_dir: Path) -> List[str]:
        filenames = []
        for page in doc.pictures:
            # print(page)
            page_no = page.self_ref.split('/')[-1]
            page_image_filename = output_dir / f"{page_no}.png"
            print(page_image_filename)
            with page_image_filename.open("wb") as fp:
                page.image.pil_image.save(fp, format="PNG")
            filenames.append(page_image_filename.relative_to(self.base_output_dir))
        return filenames 
    
    def generate_markdown(self, concepts: List[Concept]) -> None:
        """
        Generate a folder for each concept, with the images captured and a ``document.md`` file.
        
        :param concepts: ``List[Concept]`` List of concepts, their pages must in crescent order and sequentially
        (e.g. Chapter 1, 2, 3...).
        """
        for idx in tqdm_notebook(range(len(concepts))):
            curr_chap: Concept = concepts[idx]
            init_page = curr_chap.page_start
            chap_name = normalize_filename(curr_chap.concept_name)

            output_concept_dir = self.base_output_dir / chap_name
            os.makedirs(output_concept_dir, exist_ok=True)
            
            next_page = self.last_page + 1 if idx == len(concepts) - 1 else concepts[idx + 1].page_start - 1

            doc = self.document_converter.convert(self.input_doc_path, page_range=[init_page, next_page]).document
            md_str = doc.export_to_markdown()

            img_filenames = self.save_images(doc, output_concept_dir)
            raw_markdown = self._replace_image_placeholders(md_str, img_filenames)

            with open(output_concept_dir / "document.md", "w") as f:
                f.write(raw_markdown)

``EBookExtractor`` é a classe principal, encapsulando a classe criada anteriormente e servindo como uma interface de mais alto nível.

In [8]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.documents.base import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

class EbookExtractor():
    """
    Extract data from an ebook.
    
    :param pdf_file_path: The path of the desired pdf book.
    :param base_output_dir: Directory where the book is going to be saved.
    """
    def __init__(self, pdf_file_path: Path, base_output_dir: Path, llm: BaseChatModel):
        self.pages: List[Document] = None
        self.llm = llm
        self.pdf_conversor: PDFConversor = PDFConversor(pdf_file_path, base_output_dir)
        self.file_path: Path = pdf_file_path
        self._load_pdf_pages()
    
    def _load_pdf_pages(self) -> None:
        """Loads PDF and returns a list of Document objects (one per page)."""
        print(f"Loading PDF: {self.file_path}...")
        loader = PyMuPDFLoader(self.file_path)
        pages = loader.load()
        last_page = len(pages)
        print(f"Loaded {len(pages)} pages.")
        self.pages = pages
        
    
    def extract_toc_structure(self, end_toc_page = 5) -> PageExtraction:
        """
        Scans the first ``end_toc_page`` pages to find a Table of Contents or Summary.
        Returns a list of 'known concepts' to prime the main extractor.

        :param end_toc_page: The first pages where the summary appears. Default to 5.
        """
        print(f"Scouting Table of Contents (Pages 1-{end_toc_page})...")
        
        # Combine first pages (or fewer if small doc)
        limit = min(len(self.pages), end_toc_page)
        toc_text = "\n".join([p.page_content for p in self.pages[:limit]])
        
        # Simple chain for ToC extraction
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an expert content analyzer. Look at the beginning of this book."),
            ("human", """Identify the Table of Contents. 
            Extract ALL chapters, sections, and sub-sections (e.g., 1.1, 1.2.1, 1.2.2) as individual Concepts.
            Do NOT summarize or skip detailed sub-topics. Capture the full hierarchy. Do NOT include chapter number on
            concept_name.
            
            Text:
            {text}""")
        ])
        
        # We reuse the PageExtraction model, though we only care about names/start pages here
        chain = prompt | self.llm.with_structured_output(PageExtraction)
    
        try:
            result = chain.invoke({"text": toc_text})
            print(f"ToC Analysis found {len(result.concepts)} potential concepts.")
            return result.concepts
        except Exception as e:
            print(f"Could not extract ToC (might be missing or unstructured). Proceeding with empty seed. Error: {e}")
            return []
    

Executando o pipeline de extração.

In [9]:
llm = get_llm(LLM_MODELS.gpt5_1)
extractor = EbookExtractor(PDF_PATH, base_output_dir, llm)
toc_concepts = extractor.extract_toc_structure(end_toc_page=5)

Loading PDF: ../data/701-LinuxFundamentals_material_full_v14.pdf...
Loaded 127 pages.
Scouting Table of Contents (Pages 1-5)...


2025-12-10 14:04:42,068 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ToC Analysis found 104 potential concepts.


Filtrando apenas os capítulos, assim podemos gerar uma pasta para cada, contendo arquivo markdown e imagens.

In [10]:
chapters = []
for c in toc_concepts:
    if len(c.chapter) == 1:
        chapters.append(c)
        print(c.chapter, c.concept_name, c.page_start)

[1] Introdução ao Linux 6
[2] Certificações Linux 13
[3] História do Linux 16
[4] Licenças Open Source 20
[5] Evolução do Linux: distribuições 23
[6] Conhecendo o Linux 34
[7] Tópicos para revisão do capítulo 41
[8] Estrutura do sistema operacional 43
[9] O que é um Shell 52
[10] Variáveis 55
[11] Arquivos de configuração do shell 62
[12] Caminhos de Diretorios 68
[13] Tópicos para revisão do capítulo 74
[14] Como obter ajuda 76
[15] Formas de documentação 77
[16] Comando help 79
[17] Comando apropos 81
[18] Comando whatis 84
[19] Comando man 86
[20] Comando info 89
[21] Comando whereis 91
[22] Comando which 94
[23] FHS, Hierarquia dos Diretórios 96
[24] Aprendendo Comandos do GNU/Linux 110
[25] Localização no sistema 120
[26] Tópicos para revisão do capítulo 127


In [11]:
# extractor.pdf_conversor.generate_markdown(chapters)

# Gerando grafos

In [12]:
ROOT_DIRECTORY = base_output_dir
NODES_XML = ROOT_DIRECTORY / 'nodes.xml'
RELATIONS_XML = ROOT_DIRECTORY / 'relations.xml'

In [13]:
class GlobalRegistry:
    def __init__(self):
        # Stores simple strings: {"Binary Notation", "Kernel", "File Permissions"}
        self.known_concepts = set()

    def add_concepts(self, concepts: List[str]):
        for c in concepts:
            self.known_concepts.add(c)
    
    def get_context_string(self):
        """Returns a comma-separated string of known concepts for the prompt."""
        return ", ".join(sorted(list(self.known_concepts)))

In [69]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
 
class ConceptValidatorAgent:
    """
    A specialized agent that filters lists of relations based on strict 
    Knowledge Graph quality criteria.
    """
    def __init__(self, llm: BaseChatModel):
        self.received_count = 0
        self.rejected_count = 0
        self.valid_count = 0
        self.llm = llm

    def validate(self, relations: List[Relation]) -> List[Relation]:
        """
        Filters the list of relations.
        
        :param relations: List of candidate relations found by the extraction agent.
        # :param context_node: The name of the current chapter/section being analyzed.
        :return: A cleaned list of relations.
        """
        # Quick exit if empty
        if not relations:
            return []

        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a Strict Quality Control Agent for an Educational Knowledge Graph.
            
            Your Goal: Review the 'Prerequisite Relations' extracted from a text'.
            Filter out noise to ensure high-quality graph nodes.
            
            **ACCEPTANCE CRITERIA**:
            1. **Correlation**: The concepts must strong related, such that someone would struggle to learn the source
            concept without understand the target  
            """),
            ("human", "Candidate relations: {relations}")
        ])

        chain = prompt | self.llm.with_structured_output(ValidationResult)
        
        # try:
        self.received_count += len(relations)
        # We join the list to a string for the prompt
        result: ValidationResult = chain.invoke({
            # "context": context_node,
            "relations": "\n".join([f'{rel.source}-[PREREQUISITE]->{rel.target}' for rel in relations])
        })
        
        # Optional: Print what was rejected for debugging
        if result.rejected_relations:
            self.valid_count += len(result.valid_relations) 
            self.rejected_count += len(result.rejected_relations) 
            # print(f"[Validator] All: {", ".join([for ])}")
            print(f"[Validator] Accepted: {result.valid_relations}")
            print(f"[Validator] Rejected: {result.rejected_relations}")
        
        return result.valid_relations
            
        # except Exception as e:
        #     print(f"Validator Agent Failed: {e}. Keeping original list.")
        #     return relations # Fail open: keep concepts rather than losing data

In [113]:
from IPython.display import clear_output, display
import ipywidgets as widgets
from typing import Dict, Tuple
from tqdm.notebook import tqdm_notebook
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

class KnowledgeGraph():    
    def __init__(self, concepts: List[Concept], output_dir: Path, creator_llm: BaseChatModel, validator_llm: BaseChatModel):
        self.output_dir = output_dir
        self.nodes_path = self.output_dir / "nodes.xml"
        self.relations_path = self.output_dir / "relations.xml"
        
        self.root_nodes = ET.Element("nodes")
        self.root_rels = ET.Element("relations")
        
        self.files = self.from_toc(concepts, True)
        print(self.files)
        
        self.llm = creator_llm
        self.creator_agent = create_agent(
            model=LLM_MODELS.gpt5_1,
            tools=[],
            middleware=[
                SummarizationMiddleware(
                    model=LLM_MODELS.gpt5_1
                )
            ],
            response_format=ConceptAnalysis
        )
        
        self.validator = ConceptValidatorAgent(llm=validator_llm)
        self.registry = GlobalRegistry()
        
        self.node_elements = self.root_nodes.findall("node")
        self.node_elements.sort(key=lambda x: int(x.get("order", 0)))
        
        self.existing_node_names = {node.get("name").lower().strip() for node in self.node_elements}
        self.existing_node_ids = {node.get("id") for node in self.node_elements}

        print(f"Loaded {len(self.node_elements)} concepts in logical order.")

   
    def from_toc(self, concepts: List[Concept], create_relations=True):
        """
        Populates root_nodes and root_rels based on the Table of Contents hierarchy.
        """
        print(f"Building graph from {len(concepts)} ToC concepts...")
        
        hierarchy_map: Dict[Tuple[int, ...], str] = {}

        # First pass: Create Nodes
        files = {}
        folder_name = ''
        folder_count = 0
        for idx, concept in enumerate(concepts):
            if len(concept.chapter) == 1:
                folder_count += 1
                folder_name = normalize_filename(concept.concept_name)
                files[(folder_name, folder_count)] = []
                
            node_id = normalize_filename(concept.concept_name)
            files[(folder_name, folder_count)].append(concept.concept_name)
            # print(folder_name)
            
            hierarchy_map[tuple(concept.chapter)] = node_id

            node = ET.SubElement(self.root_nodes, "node")
            node.set("id", node_id)
            node.set("name", concept.concept_name)
            node.set("folder", folder_name) # Assumes folder matches ID/safe_name
            node.set("order", str(idx))
            node.set("level", str(len(concept.chapter)))
            node.set("page_start", str(concept.page_start))

        if create_relations:
            # Second pass: Create Relations (Part-Of / Including)
            for concept in concepts:
                current_id = hierarchy_map.get(tuple(concept.chapter))
                
                if len(concept.chapter) > 1:
                    parent_key = tuple(concept.chapter[:-1])
                    parent_id = hierarchy_map.get(parent_key)
                    
                    if parent_id and current_id:
                        # Relation 1: Parent INCLUDES Child
                        rel1 = ET.SubElement(self.root_rels, "relation")
                        rel1.set("type", "contains")
                        rel1.set("source", parent_id)
                        rel1.set("target", current_id)
                        ET.SubElement(rel1, "context").text = "Structural Hierarchy (ToC)"

                        # Relation 2: Child PART-OF Parent
                        # rel2 = ET.SubElement(self.root_rels, "relation")
                        # rel2.set("type", "part-of")
                        # rel2.set("source", current_id)
                        # rel2.set("target", parent_id)
                        # ET.SubElement(rel2, "context").text = "Structural Hierarchy (ToC)" 
        return files
                    
    def _add_relation(self, source, target, rel_type, context):
        """Helper to append relations to XML."""
        # Simple deduplication check could go here if needed
        if source == target:
            return 
        
        rel = ET.SubElement(self.root_rels, "relation")
        rel.set("type", rel_type)
        rel.set("source", source)
        rel.set("target", target)
        ET.SubElement(rel, "context").text = context
             
    def _append_concept(self, analysis, node_name, node_id):
        if analysis.new_concepts:
            new_count = 0
            for concept in analysis.new_concepts:
                clean_name = concept.strip()
                clean_id = normalize_filename(clean_name)
                
                if clean_id not in self.existing_node_ids:
                    # Create new Node entry
                    new_node = ET.SubElement(self.root_nodes, "node")
                    new_node.set("id", clean_id)
                    new_node.set("name", clean_name)
                    new_node.set("type", "extracted")
                    new_node.set("found_in_chapter", node_id)
                    
                    self.existing_node_names.add(clean_name.lower())
                    self.existing_node_ids.add(clean_id)
                    new_count += 1
        
                # self._add_relation(node_id, clean_id, "including", f"Extracted from {node_name}")
                # self._add_relation(clean_id, node_id, "part-of", f"Extracted from {node_name}")
                
            self.registry.add_concepts(analysis.new_concepts)
            print(f"Learned: {len(analysis.new_concepts)} concepts ({new_count} new to XML)")
            
            # The current node itself is now 'known'
            self.registry.add_concepts([node_name])
            
    def _append_relation(self, analysis: ConceptAnalysis):
        print(f'appending relations {analysis.relations}')
        for sem_rel in analysis.relations:
            # target_id = normalize_filename(sem_rel.target)
                        
            # if sem_rel.relation_type == 'prerequisite':
            self._add_relation(
                normalize_filename(sem_rel.source), 
                normalize_filename(sem_rel.target),
                "prerequisite", sem_rel.context)
            # else:
                # self._add_relation(node_id, target_id, sem_rel.relation_type, sem_rel.context) 
                
    def analyze_concept_content(self, current_concepts, text_content):
        """
        Analyzes the entire markdown content for a specific concept node.
        
        :param current_concepts: Name of the current node.
        :param text_content: Text content.
        """
        if not text_content.strip():
            return ConceptAnalysis(relations=[])

        previous_concepts_str = self.registry.get_context_string()


        prompt = ChatPromptTemplate.from_messages([
          ("system", """You are a Knowledge Graph Architect. 
           Your goal is to identify the fundamental **Prerequisite Relations** within the text.
           You can create between CURRENT CONCEPTS and PREVIOUSLY LEARNED CONCEPTS.

            CURRENT CONCEPTS:
            [{current_concepts}]
            
            ### RULES:
            1. If you are creating a prerequisite relation between current concepts, the
            target must have been taught before source.
            
            ### YOUR TASK:
            1. Check the relations:
            - **prerequisite**: Does the current concept require knowing a concept from the PREVIOUSLY LEARNED CONCEPTS list? Answer yes/no and which one.
            
            
            PREVIOUSLY LEARNED CONCEPTS:
            [{history}]
        """), 
          ("human", "{text}")
        ])

        chain = prompt | self.creator_agent
        
        try:
            analysis: ConceptAnalysis = chain.invoke({
                "current_concepts": current_concepts, 
                "text": text_content[:15000], # Safety cap for tokens
                "history": previous_concepts_str
            })['structured_response']
            
            print(f'result: {analysis}')
            
            # Validation phase
            if analysis.relations:
                # original_count = len(analysis.new_concepts_taught)
                validated_relations = self.validator.validate(
                    analysis.relations, 
                    # current_node_name
                )
                
                # Update the analysis object with the clean list
                analysis.relations = validated_relations
                
            return analysis
        except Exception as e:
            print(f"LLM Error: {e}")
            return ConceptAnalysis(relations=[])
        
        
    def build(self):
        """
        Build the graph containing the initial concepts, new concepts learned and the relation
        between them.
        """
        # output = widgets.Output()
        # display(output) 
        
        for folder, idx in tqdm_notebook(sorted(self.files.keys(), key=lambda x: x[1]),
                                    desc="Creating relations and new nodes..."):
            nodes = self.files[(folder, idx)]
            concepts = ', '.join(nodes)
            # break
            
            # Path to the granular MD file
            file_path = os.path.join(self.output_dir, folder, "document.md")
            
            if not os.path.exists(file_path):
                continue
            
            # with output:
            # output.clear_output(wait=True)
            print(f"Analyzing: {folder}")
            
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
            analysis = self.analyze_concept_content(concepts, content)
            # self._append_concept(analysis, node_name, node_id) # add concepts            
            self.registry.add_concepts(nodes) # Add relations
            self._append_relation(analysis) # Add relations
            # break
            

    def save(self):
        """
        Save the final XML files.
        """
        xml_rels_str = prettify_xml(self.root_rels)
        with open(self.relations_path, "w", encoding="utf-8") as f:
            f.write(xml_rels_str)
            
        xml_nodes_str = prettify_xml(self.root_nodes)
        with open(self.nodes_path, "w", encoding="utf-8") as f:
            f.write(xml_nodes_str)
            
        print(f"Updated {self.nodes_path} and {self.relations_path}")

In [114]:
creator_llm = get_llm(LLM_MODELS.gpt5_1)
validator_llm = get_llm(LLM_MODELS.gpt5_1)
kg = KnowledgeGraph(toc_concepts, base_output_dir, creator_llm, validator_llm)

Building graph from 104 ToC concepts...
{('introducao_ao_linux', 1): ['Introdução ao Linux', 'Sobre o material', 'Mercado Linux', 'Onde podemos encontrar o Linux?', 'Market Share – Sistemas Operacionais pelo mundo', 'Dispositivos móveis', 'Desktop Linux', 'Servidores web', 'Profi ssões no mundo OpenSource'], ('certificacoes_linux', 2): ['Certificações Linux', 'Centros certificadores'], ('historia_do_linux', 3): ['História do Linux', 'A origem do Linux'], ('licencas_open_source', 4): ['Licenças Open Source', 'Free Software Foundation', 'Software livre', 'GNU GPL (General Public License)'], ('evolucao_do_linux_distribuicoes', 5): ['Evolução do Linux: distribuições', 'Distribuições livres e corporativas', 'Distribuições from scratch e derivadas', 'Ciclo de vida de algumas distribuições Linux', 'Debian GNU/Linux', 'Evolução do Linux: dispositivos embarcados'], ('conhecendo_o_linux', 6): ['Conhecendo o Linux', 'Principais aplicações desktop Open-Source', 'Principais aplicações em servidores

In [115]:
kg.build()

Creating relations and new nodes...:   0%|          | 0/26 [00:00<?, ?it/s]

Analyzing: introducao_ao_linux


2025-12-10 15:16:38,229 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


result: relations=[]
appending relations []
Analyzing: certificacoes_linux


2025-12-10 15:16:50,306 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

[Validator] Accepted: [Relation(source='Comando find', target='Curingas', relation_type='prerequisite', context='Para usar o comando find de forma eficaz é importante entender curingas (wildcards) para especificar padrões de nomes de arquivos.'), Relation(source='Comando find', target='Estrutura de Diretórios GNU/Linux', relation_type='prerequisite', context='Para localizar arquivos com find é necessário compreender a estrutura de diretórios do GNU/Linux, pois a busca é feita em diretórios específicos.'), Relation(source='Comando find', target='Diretórios /home e /root', relation_type='prerequisite', context='Saber o papel dos diretórios /home e /root ajuda a entender onde buscar arquivos de usuários e do administrador com find.'), Relation(source='Comando find', target='Diretório /', relation_type='prerequisite', context='Entender o diretório raiz / é essencial para saber o ponto de partida de buscas globais com find.'), Relation(source='Comando find', target='Diretório /etc', relatio

2025-12-10 15:26:25,564 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


result: relations=[Relation(source='Tópicos para revisão do capítulo', target='Estrutura de Diretórios GNU/Linux', relation_type='prerequisite', context='Para revisar tópicos sobre diretórios organizados segundo o FHS e comandos que atuam sobre arquivos/diretórios, é necessário já conhecer a estrutura de diretórios do GNU/Linux.'), Relation(source='Tópicos para revisão do capítulo', target='FHS', relation_type='prerequisite', context='O primeiro tópico de revisão cita explicitamente que os diretórios são organizados segundo o padrão FHS, então é preciso já ter aprendido o conceito de FHS.'), Relation(source='Tópicos para revisão do capítulo', target='Comando find', relation_type='prerequisite', context='Um dos tópicos revisa o comando find; para entendê-lo em revisão, o comando find já deve ter sido apresentado antes.'), Relation(source='Tópicos para revisão do capítulo', target='Comando xargs', relation_type='prerequisite', context='Há um item de revisão sobre a função do comando xarg

2025-12-10 15:26:28,692 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Validator] Accepted: []
[Validator] Rejected: [Relation(source='Tópicos para revisão do capítulo', target='Estrutura de Diretórios GNU/Linux', relation_type='prerequisite', context=None), Relation(source='Tópicos para revisão do capítulo', target='FHS', relation_type='prerequisite', context=None), Relation(source='Tópicos para revisão do capítulo', target='Comando find', relation_type='prerequisite', context=None), Relation(source='Tópicos para revisão do capítulo', target='Comando xargs', relation_type='prerequisite', context=None), Relation(source='Tópicos para revisão do capítulo', target='Comando locate', relation_type='prerequisite', context=None)]
appending relations []


In [116]:
kg.save()

Updated ebooks/LinuxFundamentals/nodes.xml and ebooks/LinuxFundamentals/relations.xml


In [117]:
OUTPUT_HTML = base_output_dir / 'graph.html'
OUTPUT_HTML = str(OUTPUT_HTML)

In [119]:
import os
import xml.etree.ElementTree as ET
from pyvis.network import Network
import networkx as nx

def get_node_color_by_level(level_str):
    """Returns a color hex code based on hierarchy level."""
    try:
        level = int(level_str)
    except (ValueError, TypeError):
        level = 1

    # Palette: Deep Blue -> Teal -> Green -> Light Green
    palette = {
        0: "#f0a202", # Gold (Root)
        1: "#22577a", # Dark Blue (Chapters)
        2: "#38a3a5", # Teal (Sections)
        3: "#57cc99", # Mint (Sub-sections)
        4: "#80ed99", # Light Green
        5: "#c7f9cc"  # Pale Green
    }
    return palette.get(level, "#c7f9cc") # Default to lightest for deep levels

def create_interactive_graph():
    if not os.path.exists(NODES_XML) or not os.path.exists(RELATIONS_XML):
        print("❌ XML files not found.")
        return

    print("📊 Constructing NetworkX Graph...")
    G = nx.DiGraph()

    # 1. Parse Nodes
    tree_nodes = ET.parse(NODES_XML)
    for node in tree_nodes.findall("node"):
        node_id = node.get("id")
        name = node.get("name")
        node_type = node.get("type", "chapter") 
        level = node.get("level", "1")
        
        # Default styling
        size = 15
        title = f"Type: {node_type}\nLevel: {level}"
        
        if node_type == "extracted":
            color = "#ffafcc" # Pink/Pastel for Extracted Concepts (Distinct from structure)
            size = 10         
            found_in = node.get("found_in_chapter", "unknown")
            title += f"\nFound in: {found_in}"
        elif node_type == "root":
            color = get_node_color_by_level(0)
            size = 30
        else:
            # Structural Node (Chapter/ToC) -> Color by Level
            color = get_node_color_by_level(level)
            # Make higher levels slightly bigger
            size = max(10, 25 - (int(level) * 3))

        G.add_node(node_id, label=name, title=title, color=color, size=size, shape="dot")

    # 2. Parse Relations
    tree_rels = ET.parse(RELATIONS_XML)
    for rel in tree_rels.findall("relation"):
        source = rel.get("source")
        target = rel.get("target")
        rel_type = rel.get("type")
        
        # --- EDGE STYLING LOGIC ---
        # Default
        color = "#888888" 
        width = 1
        dashes = False
        
        if rel_type == "prerequisite":
            color = "#ff3366"  # Bright Red/Pink (Critical Path)
            width = 3
            dashes = False
            
        elif rel_type == "part-of":
            color = "#4a90e2"  # Solid Blue
            width = 1
            dashes = True      # Dashed to show hierarchy
            
        elif rel_type == "including":
            color = "#00b4d8"  # Cyan
            width = 1
            dashes = True
            
        elif rel_type == "definition":
            color = "#9b5de5"  # Purple (Semantic Definition)
            width = 2
            dashes = False
            
        elif rel_type == "property":
            color = "#f15bb5"  # Magenta (Attribute/Property)
            width = 1
            dashes = False

        if G.has_node(source) and G.has_node(target):
            # Title allows you to hover over the edge to see "prerequisite", etc.
            G.add_edge(source, target, title=rel_type, color=color, width=width, dashes=dashes)

    print(f"🕸️  Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

    # 3. Generate PyVis Visualization (Obsidian Style)
    print("🎨 Generating Obsidian-like HTML Visualization...")
    
    net = Network(height="900px", width="100%", bgcolor="#1e1e1e", font_color="#cccccc", select_menu=False, filter_menu=False)
    
    net.from_nx(G)
    
    options = """
    var options = {
      "nodes": {
        "borderWidth": 0,
        "borderWidthSelected": 2,
        "font": {
          "size": 14,
          "face": "tahoma",
          "color": "#eeeeee",
          "strokeWidth": 2,
          "strokeColor": "#1e1e1e"
        },
        "shadow": {
            "enabled": true,
            "color": "black",
            "size": 5,
            "x": 2,
            "y": 2
        }
      },
      "edges": {
        "smooth": {
          "type": "continuous",
          "forceDirection": "none"
        },
        "arrows": {
            "to": {
                "enabled": true,
                "scaleFactor": 0.5
            }
        },
        "color": {
            "inherit": false,
            "opacity": 1.0
        }
      },
      "interaction": {
        "hover": true,
        "hoverConnectedEdges": true,
        "selectConnectedEdges": true,
        "navigationButtons": true,
        "keyboard": true,
        "tooltipDelay": 200
      }
    }
    """
    net.set_options(options)
    
    # net.show_buttons(filter_=['physics'])

    net.save_graph(OUTPUT_HTML)
    print(f"✅ Visualization saved to: {os.path.abspath(OUTPUT_HTML)}")


In [120]:
create_interactive_graph()

📊 Constructing NetworkX Graph...
🕸️  Graph created with 101 nodes and 206 edges.
🎨 Generating Obsidian-like HTML Visualization...
✅ Visualization saved to: /home/pras/EMBRAPII/4linux/notebooks/ebooks/LinuxFundamentals/graph.html


In [28]:
import os
import xml.etree.ElementTree as ET
from collections import Counter
from typing import List, Tuple, Dict, Any

def get_single_run_stats(nodes_path: str, relations_path: str) -> Dict[str, Any]:
    """
    Helper function to parse a single pair of XML files and return raw stats.
    """
    stats = {
        "status": "OK",
        "nodes_total": 0,
        "nodes_chapter": 0,
        "nodes_extracted": 0,
        "relations_total": 0,
        "rel_prereq": 0,
        "rel_partof": 0,
        "rel_def": 0,
        "rel_prop": 0,
        "orphans": 0,
        "avg_rel": 0.0
    }

    if not os.path.exists(nodes_path) or not os.path.exists(relations_path):
        stats["status"] = "Missing Files"
        return stats

    try:
        # 1. Analyze Nodes
        tree_nodes = ET.parse(nodes_path)
        root_nodes = tree_nodes.getroot()
        all_nodes = root_nodes.findall("node")
        
        stats["nodes_total"] = len(all_nodes)
        node_ids = set()

        for node in all_nodes:
            n_type = node.get("type", "chapter")
            if n_type == "extracted":
                stats["nodes_extracted"] += 1
            else:
                stats["nodes_chapter"] += 1 # Counts root and chapters together
            
            node_ids.add(node.get("id"))

        # 2. Analyze Relations
        tree_rels = ET.parse(relations_path)
        root_rels = tree_rels.getroot()
        all_rels = root_rels.findall("relation")
        
        stats["relations_total"] = len(all_rels)
        
        connected_nodes = set()

        for rel in all_rels:
            r_type = rel.get("type", "unknown")
            
            if r_type == "prerequisite":
                stats["rel_prereq"] += 1
            elif r_type == "part-of":
                stats["rel_partof"] += 1
            elif r_type == "definition":
                stats["rel_def"] += 1
            elif r_type == "property":
                stats["rel_prop"] += 1
            
            connected_nodes.add(rel.get("source"))
            connected_nodes.add(rel.get("target"))

        # 3. Health Metrics
        orphans = node_ids - connected_nodes
        stats["orphans"] = len(orphans)
        
        if stats["nodes_total"] > 0:
            stats["avg_rel"] = round(stats["relations_total"] / stats["nodes_total"], 2)

    except Exception as e:
        stats["status"] = f"Error: {str(e)[:20]}..."
    
    return stats

def analyze_graph_files(graph_runs):
    headers = [
        "Generator", "Validator", "Status", "Nodes", "Chapters", "Concepts",
        "Relations", "Prereqs", "Part-Of", "Defs", "Props", "Orphans", "Avg Rel/Node"
    ]
    lines = []
    lines.append("### Knowledge Graph Analysis Report")
    header_row = "| " + " | ".join(headers) + " |"
    separator = "| " + " | ".join(["---"] * len(headers)) + " |"
    lines.append(header_row)
    lines.append(separator)
    

    for idx, (run_name, n_path, r_path) in enumerate(graph_runs, 1):
        data = get_single_run_stats(n_path, r_path)
        if data["status"] != "OK":
            row = [str(idx), data["status"]] + ["-"] * (len(headers) - 2)
        else:
            row = [
                str(run_name.split('_')[0]),
                str(run_name.split('_')[1]),
                "✅ OK",
                str(data["nodes_total"]),
                str(data["nodes_chapter"]),
                str(data["nodes_extracted"]),
                str(data["relations_total"]),
                str(data["rel_prereq"]),
                str(data["rel_partof"]),
                str(data["rel_def"]),
                str(data["rel_prop"]),
                str(data["orphans"]),
                str(data["avg_rel"])
            ]
        lines.append("| " + " | ".join(row) + " |")
    lines.append("\n*Note: 'Chapters' includes structural nodes (Root, Sections, Chapters). 'Concepts' are LLM extracted entities.*")
    return "\n".join(lines)

In [72]:
import os
from pathlib import Path

GRAPH_RUNS_DIR = Path("graph_runs")
graph_runs = []

for run_dir in GRAPH_RUNS_DIR.iterdir():
    if run_dir.is_dir():
        nodes_path = run_dir / "nodes.xml"
        relations_path = run_dir / "relations.xml"
        if nodes_path.exists() and relations_path.exists():
            graph_runs.append((run_dir.name, nodes_path, relations_path))
graph_runs

[('claude-opus-4-5_gpt5-1',
  PosixPath('graph_runs/claude-opus-4-5_gpt5-1/nodes.xml'),
  PosixPath('graph_runs/claude-opus-4-5_gpt5-1/relations.xml')),
 ('claude-opus-4-5_claude-opus-4-5',
  PosixPath('graph_runs/claude-opus-4-5_claude-opus-4-5/nodes.xml'),
  PosixPath('graph_runs/claude-opus-4-5_claude-opus-4-5/relations.xml')),
 ('gpt5-1_gpt5-1',
  PosixPath('graph_runs/gpt5-1_gpt5-1/nodes.xml'),
  PosixPath('graph_runs/gpt5-1_gpt5-1/relations.xml')),
 ('gpt5-1_claude-opus-4-5',
  PosixPath('graph_runs/gpt5-1_claude-opus-4-5/nodes.xml'),
  PosixPath('graph_runs/gpt5-1_claude-opus-4-5/relations.xml'))]